In [76]:
from scripts.import_data import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [61]:
import pandas as pd
import xlrd
import os
import warnings

file_paths = ["Rocznik_2014__GR.xls", "Rocznik_2015__GR.xls", "Rocznik_2016__GR.xls", "Rocznik_2017_GR.xls", "Rocznik_2018_GR.xls", "Rocznik_2019_GR.xls",
              "Rocznik_2020_GR.xls", "Rocznik_2021_GR.xls", "Rocznik_2022_GR.xls"]

# map files to tab names [tabs are mixed between years]
files_to_tab_names = dict()
headers = dict()
for ind, path in enumerate(file_paths):
  files_to_tab_names[path] = dict()
  headers[path] = dict()

  if ind <= 3:
    files_to_tab_names[path]["market_value"] = "Tab 17"
  else:
    files_to_tab_names[path]["market_value"] = "Tab 12"

In [62]:
import pandas as pd
import os

def read_excel_files(path):
    excel_data = {}
    for file in os.listdir(path):
        if file.endswith(".xls"):
            year = file.split('_')[1]  # Extracting year from file name
            xls = pd.ExcelFile(os.path.join(path, file))
            for sheet_name in xls.sheet_names:
                if sheet_name == files_to_tab_names[file]["market_value"]:
                    df = pd.read_excel(xls, sheet_name=sheet_name, header=3)
                    for index, row in df.iterrows():
                      company = row[1]
                      if company not in excel_data:
                        excel_data[company] = {}
                      if int(year) >= 2018:
                        excel_data[company][year] = row[5]
                      else:
                        excel_data[company][year] = row[3]

    return excel_data

excel_data = read_excel_files("./")
excel_data = {key: dict(sorted(value.items(), reverse=True)) for key, value in excel_data.items()}
for company, data in excel_data.items():
  for year, value in data.items():
   if str(int(year) - 1) in excel_data[company].keys() and excel_data[company][str(int(year) - 1)] != 0:
      excel_data[company][year] = 100*(value - excel_data[company][str(int(year) - 1)])/excel_data[company][str(int(year) - 1)]

# Create DataFrame
df = pd.DataFrame.from_dict(excel_data, orient='index')
print(df)
df.to_csv('market_values.csv', index=True)

WARNING *** file size (1183124) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
                2022       2021       2020          2019       2018  \
PKOBP      56.441504 -16.656994 -12.693185    -10.923042  57.462687   
PKNORLEN   28.794157 -32.742939 -20.647244      2.028292  24.267301   
PZU         9.239826 -19.160652  -8.815479      4.127135  26.949710   
SANPL      87.668260 -39.530316 -14.182033  36568.030000        NaN   
CDPROJEKT -29.719242   2.919316  91.964313     50.103071  85.823903   
...              ...        ...        ...           ...        ...   
GLOBALNRG        NaN        NaN        NaN           NaN        NaN   
GREENECO         NaN        NaN        NaN           NaN        NaN   
ENERGOPLD        NaN        NaN        NaN           NaN        NaN   
CCIINT           NaN        NaN        NaN           NaN        NaN   
NOVAKBM          NaN        NaN        NaN           NaN        NaN   

  

In [94]:
import copy

market_values = pd.read_csv('./Data/market_values.csv')
market_values.set_index('Unnamed: 0', inplace=True)
print(market_values.index)
print(market_values)

Index([    'PKOBP',  'PKNORLEN',       'PZU',     'SANPL', 'CDPROJEKT',
           'PEKAO',    'INGBSK',     'PGNIG',      'KGHM', 'CYFRPLSAT',
       ...
         'COMPLEX', 'MOSTALEXP',    'MISPOL',   'IDEATFI', 'BUDVARCEN',
       'GLOBALNRG',  'GREENECO', 'ENERGOPLD',    'CCIINT',   'NOVAKBM'],
      dtype='object', name='Unnamed: 0', length=633)
                 2022       2021       2020          2019       2018  \
Unnamed: 0                                                             
PKOBP       56.441504 -16.656994 -12.693185    -10.923042  57.462687   
PKNORLEN    28.794157 -32.742939 -20.647244      2.028292  24.267301   
PZU          9.239826 -19.160652  -8.815479      4.127135  26.949710   
SANPL       87.668260 -39.530316 -14.182033  36568.030000        NaN   
CDPROJEKT  -29.719242   2.919316  91.964313     50.103071  85.823903   
...               ...        ...        ...           ...        ...   
GLOBALNRG         NaN        NaN        NaN           NaN        NaN   

In [98]:
import copy

balance_data = import_csv_dict_of_dataframes('./Data/companies_balance_sheet.csv')
for company, years in balance_data.items():
    for year, data in years.items():
        if company in market_values.index and year in market_values.columns:
            # Take only the second level of the MultiIndex
            data.index = data.index.droplevel(0)

#in progress - zmiana na procenty

#balance_data2 = copy.deepcopy(balance_data)
# for company, years in balance_data.items():
#     for year, data in years.items():
#         if company in df.index and year in df.columns:
#             for indicator, value in data.items():
#               if str(int(year)-1) in balance_data2[company].keys() and isinstance(balance_data[company][year][indicator], float):
#                 balance_data[company][year][indicator] = (value - balance_data2[company][str(int(year)-1)])[indicator]/balance_data2[company][str(int(year)-1)][indicator]

columns = ['Accounting period', 'Revenues from sales', 'Operating profit/loss',
       'Net profit/loss attributable to equity holders of the parent',
       'Total comprehensive income attributable to equity holders',
       'Depreciation', 'Cash flow from operating activities',
       'Cash flow from investing activities',
       'Cash flow from financing activities', 'Aktywa', 'Non-current assets',
       'Current assets', 'Equity shareholders of the parent',
       'Non-current liabilities', 'Current liabilities', 'Earnings per share',
       'Total comprehensive income per share', 'Book value per share', 'market_value']

rows = []
for company, years in balance_data.items():
    for year, market_data in years.items():
        new_row = {}
        for indicator, value in market_data.items():
            if indicator in columns:  # Check if the indicator exists in columns
                new_row[indicator] = value
        if company in market_values.index and year in market_values.columns:
            new_row['market_value'] = market_values[year][company]
            rows.append(new_row)

# Create DataFrame from the list of rows
data = pd.DataFrame(rows, columns=columns)
print(data)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
('TAURON', 'Net profit/loss attributable to equity holders of the parent')
('TAURON', 'Total comprehensive income attributable to equity holders')
('TAURON', 'Depreciation')
('TAURON', 'Cash flow from operating activities')
('TAURON', 'Cash flow from investing activities')
('TAURON', 'Cash flow from financing activities')
('TAURON', 'Aktywa')
('TAURON', 'Non-current assets')
('TAURON', 'Current assets')
('TAURON', 'Equity shareholders of the parent')
('TAURON', 'Non-current liabilities')
('TAURON', 'Current liabilities')
('TAURON', "Number of shares ('000)")
('TAURON', 'Earnings per share')
('TAURON', 'Total comprehensive income per share')
('TAURON', 'Book value per share')
('TAURON', 'Accounting standard')
('TAURON', 'Currency')
('TAURON', 'Note')
('TAURON', 'Accounting period')
('TAURON', 'Accounting period')
('TAURON', 'Basic information')
('TAURON', 'End of period')
('TAURON', 'Start of period')
('TAURON', 'Revenues 

In [99]:
data = data.dropna(subset=data.columns[data.isna().any()])
X = data.drop('market_value', axis=1)
y = data['market_value']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Support Vector Machine': SVR(kernel='linear'),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'{name} MSE: {mse}')

# Pick the best performing model for deployment


Linear Regression MSE: 213597.32310560328
Random Forest MSE: 211449.37872596472
Gradient Boosting MSE: 215329.94994151284
